<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/EVAL_NEMO_TOOLKIT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/NVIDIA/NeMo-Agent-Toolkit/blob/develop/docs/source/improve-workflows/evaluate.md

In [1]:
#!git clone https://github.com/NVIDIA/NeMo-Agent-Toolkit.git

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/tools/NeMo-Agent-Toolkit/examples/evaluation_and_profiling/

email_phishing_analyzer  simple_web_query_eval
simple_calculator_eval	 swe_bench


In [4]:
!mkdir -p /content/NeMo-Agent-Toolkit/examples/

In [5]:
!cp -pr  /content/drive/MyDrive/tools/NeMo-Agent-Toolkit/examples/evaluation_and_profiling   /content/NeMo-Agent-Toolkit/examples/

In [6]:
!ls /content/NeMo-Agent-Toolkit/examples/evaluation_and_profiling/

email_phishing_analyzer  simple_web_query_eval
simple_calculator_eval	 swe_bench


In [ ]:
!pip install nvidia-nat -q

In [ ]:
!uv pip install "nvidia-nat[all]"

In [9]:
!nat --version

nat, version 1.3.1


In [10]:
from google.colab import userdata
api_key = userdata.get('NVIDIA_API_KEY')
import os
os.environ['NVIDIA_API_KEY'] = api_key

api_key_TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = api_key_TAVILY_API_KEY

In [11]:
!cat /content/drive/MyDrive/data/workflow.yml

functions:
   # Add a tool to search wikipedia
   wikipedia_search:
      _type: wiki_search
      max_results: 2

llms:
   # Tell NeMo Agent Toolkit which LLM to use for the agent
   nim_llm:
      _type: nim
      model_name: meta/llama-3.1-70b-instruct
      temperature: 0.0

workflow:
   # Use an agent that 'reasons' and 'acts'
   _type: react_agent
   # Give it access to our wikipedia search tool
   tool_names: [wikipedia_search]
   # Tell it which LLM to use
   llm_name: nim_llm
   # Make it verbose
   verbose: true
   # Retry up to 3 times
   parse_agent_response_max_retries: 3

In [12]:
%cd /content/NeMo-Agent-Toolkit/

/content/NeMo-Agent-Toolkit


## Evaluating a Workflow

In [ ]:
import pygame
# Initialize only what you need (e.g., fonts for text processing)
pygame.font.init()
# Do NOT call pygame.init() or pygame.mixer.init()

import os
# Force SDL to use a dummy video and audio driver
os.environ['SDL_VIDEODRIVER'] = 'dummy'
os.environ['SDL_AUDIODRIVER'] = 'dummy'

# Fix the XDG_RUNTIME_DIR warning by creating a temp directory
os.environ['XDG_RUNTIME_DIR'] = '/tmp/runtime-root'
if not os.path.exists('/tmp/runtime-root'):
    os.makedirs('/tmp/runtime-root', mode=0o700)


!export XDG_RUNTIME_DIR=/tmp/runtime-root && mkdir -p /tmp/runtime-root && chmod 700 /tmp/runtime-root

In [14]:
# Quick verification - check if keys exist in the environment
nv_key = os.getenv('NVIDIA_API_KEY')
tv_key = os.getenv('TAVILY_API_KEY')

if nv_key and tv_key:
    print("✅ Success: Both NVIDIA and Tavily keys are successfully set in the environment.")
else:
    if not nv_key: print("❌ Error: NVIDIA_API_KEY is missing.")
    if not tv_key: print("❌ Error: TAVILY_API_KEY is missing.")

✅ Success: Both NVIDIA and Tavily keys are successfully set in the environment.


In [15]:
!nat eval --config_file=/content/NeMo-Agent-Toolkit/examples/evaluation_and_profiling/simple_web_query_eval/configs/eval_config.yml

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
2026-01-20 03:35:02 - WARNING  - py.warnings:112 - /usr/local/lib/python3.12/dist-packages/moviepy/video/io/sliders.py:61: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if event.key is 'enter':

2026-01-20 03:

In [16]:
import json
import pandas as pd

# Path to your results
results_path = ".tmp/nat/examples/evaluation_and_profiling/simple_web_query_eval/eval/accuracy_output.json"

with open(results_path, 'r') as f:
    data = json.load(f)

# NAT 1.3 uses 'eval_output_items' for individual entries
if 'eval_output_items' in data:
    # Use json_normalize to flatten the nested 'reasoning' dictionary automatically
    df = pd.json_normalize(data['eval_output_items'])

    print(f"--- Global Average Accuracy: {data.get('average_score')} ---")

    # In this version, 'reasoning' fields are flattened to 'reasoning.user_input', etc.
    # We'll select the most relevant columns for display:
    cols_to_show = ['reasoning.user_input', 'reasoning.response', 'score']

    # Filter only available columns to avoid future KeyErrors
    available_cols = [c for c in cols_to_show if c in df.columns]
    print(df[available_cols].head())
else:
    print(f"❌ Key not found. Available keys: {list(data.keys())}")

--- Global Average Accuracy: 0.7 ---
                                reasoning.user_input  \
0           What are the main features of LangSmith?   
1  How does LangSmith help with LLM application o...   
2  What is the purpose of LangSmith's evaluation ...   
3     How does LangSmith support prompt engineering?   
4  Is LangSmith framework-specific or can it be u...   

                                  reasoning.response  score  
0                                                      1.00  
1  LangSmith helps with LLM application observabi...   0.50  
2  The purpose of LangSmith's evaluation features...   0.50  
3  LangSmith supports prompt engineering by provi...   0.75  
4  LangSmith is not framework-specific and can be...   0.75  


In [ ]:
!nat eval --config_file examples/evaluation_and_profiling/simple_web_query_eval/src/nat_simple_web_query_eval/configs/eval_config_llama31.yml

In [24]:
import json
import pandas as pd

# Path to your NAT 1.3 accuracy results
results_path = ".tmp/nat/examples/evaluation_and_profiling/simple_web_query_eval/llama-31-8b/accuracy_output.json"

with open(results_path, 'r') as f:
    data = json.load(f)

# Flattening the NAT 1.3 'eval_output_items' structure
if 'eval_output_items' in data:
    # Use json_normalize to flatten the nested reasoning fields
    df = pd.json_normalize(data['eval_output_items'])

    # Selecting the most useful columns for review
    # user_input: The question
    # reference: The 'Gold Standard' answer from your dataset
    # response: What your agent actually replied
    display_df = df[[
        'reasoning.user_input',
        'reasoning.reference',
        'reasoning.response',
        'score'
    ]].copy()

    # Set display options for easier reading
    pd.set_option('display.max_colwidth', 500)

    print(f"--- Global Average Accuracy: {data.get('average_score')} ---")
    display(display_df.sort_values(by='score')) # Sorting by lowest score to see failures first
else:
    print(f"❌ Could not find 'eval_output_items'. Available keys: {list(data.keys())}")

--- Global Average Accuracy: 0.75 ---


,reasoning.user_input,reasoning.reference,reasoning.response,score
0,What are the main features of LangSmith?,"LangSmith offers three main features: Observability for monitoring and analyzing traces, Evals for evaluating application performance and getting human feedback, and Prompt Engineering for iterating on prompts with version control and collaboration features.","The main features of LangSmith include:\n\n1. Tracing and Debugging: LangSmith traces the flow of data through your application, making it easier.\n2. LLM-native observability: It provides LLM-native observability that allows you to trace entire application pipelines, monitor key performance metrics in production, and gain the meaningful insights needed to debug, optimize, and improve your products.\n3. Practical workflow features: LangSmith supports practical workflow features such as the a...",0.25
2,What is the purpose of LangSmith's evaluation features?,"LangSmith's evaluation features help in building and running high-quality evaluations for AI applications. It allows you to quickly assess application performance using off-the-shelf evaluators, analyze evaluation results in the UI, compare results over time, and collect human feedback to improve the application.","The purpose of LangSmith's evaluation features is to provide a robust platform for testing, monitoring, and optimizing LLM applications, ensuring they are predictable, reliable, and meet company expectations.",0.50
1,How does LangSmith help with LLM application observability?,"LangSmith provides LLM-native observability features that help throughout all stages of application development. It allows you to add tracing to your application, create dashboards to view key metrics like RPS, error rates and costs, and get meaningful insights from your application.","LangSmith Observability provides complete visibility into agent behavior with tracing, real-time monitoring, alerting, and high-level insights into usage. Key features include:\n\n1. Tracing setup: Configure tracing with basic options, framework integrations, or advanced settings for full control.\n2. Viewing & managing traces: Access and manage traces via UI or API with filtering, exporting, sharing, and comparison tools.\n3. Monitoring performance: Create dashboards and set alerts to track...",1.00
3,How does LangSmith support prompt engineering?,"LangSmith provides tools for prompt engineering that help you find the perfect prompt for your application. You can create prompts, iterate on models and prompts using the Playground, and manage prompts programmatically in your application.",,1.00
4,Is LangSmith framework-specific or can it be used with any LLM application?,"LangSmith is framework-agnostic and can be used with or without LangChain's open source frameworks. It can be integrated with any LLM application, though it offers specific integration options for LangChain and LangGraph users through environment variables.",,1.00
